<div style="text-align: center;">

# **Text Classification for Spam**

</div>

<div align="center">
 <img src="https://storage.googleapis.com/gweb-cloudblog-publish/images/image4_v2LFcq0.max-1200x1200.png" alt="drawing" width="1000"/>
</div>


# **1. Business Objective**

## **Identify the business task and translate it into an NLP Task**

This project aims to develop a robust system for identifying spam emails, translating the business need into an NLP task. The goal is to differentiate between legitimate messages and spam, thereby reducing time wasted on malicious emails.

## **Consider key stakeholders**

- **IT Department**: Responsible for maintaining and improving email filtering systems.
- **Employees**: Will benefit from a cleaner inbox and reduced time spent dealing with spam.
- **Company Management**: Interested in improving overall productivity and reducing security risks associated with spam emails (e.g., phishing scams).

## **Objectives**

- **Obtain the dataset**: Gather a suitable dataset of email messages labeled as spam or legitimate.
- **Set up the environment and download all dependencies**: Ensure all necessary software and libraries are installed.
- **Pre-process the data**: Clean and preprocess the email data, including tasks such as removing HTML tags, stopwords, and punctuation.
- **Perform feature engineering**: Extract relevant features from the text data, such as word frequency, n-grams, and TF-IDF scores.
- **Model selection**: Choose appropriate machine learning or deep learning models for text classification, considering factors such as performance and scalability.
- **Training and testing the model**: Train the selected model on the preprocessed data and evaluate its performance using appropriate metrics.
- **Implement the final model and test on emails**: Deploy the trained model to classify incoming emails as spam or legitimate and assess its effectiveness in a real-world scenario.



# **2. Setting up the Environment**

## **Key Objectives**

- **Import Required Libraries**: Load essential libraries for data manipulation (`pandas`, `numpy`), preprocessing (`StandardScaler`, `MinMaxScaler`), feature engineering (`SelectKBest`, `PCA`), handling imbalanced data (`SMOTE`, `RandomUnderSampler`), model building (`LogisticRegression`, `RandomForestClassifier`), and evaluation (`accuracy_score`).
- **Visualization Tools**: Import visualization libraries (`seaborn`, `matplotlib`) for data analysis and presentation.
- **Install Necessary Packages**: Ensure external libraries are installed (`ucimlrepo`).
- **Environment Configuration**: Set up any required environment variables or settings.

This section ensures all necessary libraries and tools are available for data analysis, model building, and evaluation.

In [ ]:
# Setting the environment by importing all the required modules and libraries
import os
import requests
import re

# Data manipulation and preprocessing libraries and modules
import pandas as pd
import numpy as np

# Data pre-processing
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Feature engineering
from sklearn.feature_selection import SelectKBest, f_classif, RFE, SelectFromModel
from sklearn.decomposition import PCA

# Handling imbalanced data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score

# Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# Install additional libraries
!pip install ucimlrepo

# **3. System for Fetching Latest Dataset**

## **Key Objectives**

- **Reflecting Current Trends and Patterns**: Ensure the dataset reflects the most recent trends by fetching the latest version from the source repository.
- **Improved Accuracy and Performance**: Regularly update the dataset to improve model accuracy and performance by using the most up-to-date data.
- **Continuous Improvement**: Automate the data retrieval process to ensure continuous improvement and seamless updates.

This section describes the automated system for fetching and loading the latest version of the dataset to keep the analysis current and relevant.
It workes hand in hand with with the GitHub Actions workflows that has been set up to run every sunday midnight to retrieve the latest dataset

In [ ]:
import pandas as pd
import requests
import re
from ucimlrepo import fetch_ucirepo

def fetch_current_version():
    """Fetch the current dataset version from the GitHub repository."""
    url = "https://raw.githubusercontent.com/uci-ml-repo/ucimlrepo/main/setup.py"
    response = requests.get(url)
    
    if response.status_code == 200:
        version_match = re.search(r"version='(.*?)'", response.text)
        if version_match:
            return version_match.group(1)
    return None

def fetch_and_load_dataset():
    """Fetch and load the dataset."""
    spambase = fetch_ucirepo(id=94)
    
    # Convert data to DataFrames
    data_df = pd.DataFrame(spambase.data.features)
    target_df = pd.DataFrame(spambase.data.targets, columns=["Class"])
    
    # Combine features and target variable into a single DataFrame
    df = pd.concat([data_df, target_df], axis=1)
    
    return df

def main():
    """Main function to automate the data retrieval process and use the DataFrame directly."""
    current_version = fetch_current_version()
    if not current_version:
        print("Failed to fetch the current version from the repository.")
        return None
    
    print(f"Current dataset version: {current_version}")
    
    # Fetch and load the dataset
    df = fetch_and_load_dataset()
    
    # Display the first few rows of the dataset
    print(df.head())
    
    return df

# Execute the main function and load the DataFrame
df = main()

# Continue with your data processing and model training
if df is not None:
    # Example: Show the first few rows of the DataFrame
    print(df.head())

# **4. Data Exploration**
Reflecting current trends and patterns

The data exploration phase involves understanding the underlying trends and patterns in the dataset to gain insights that will guide further analysis and model development.

## Key Objectives:
* **Understand the Data Structure:** Load and inspect the dataset to get an overview of the data.
* **Data Cleaning:** Identify and handle missing values, and remove duplicates to ensure data quality.
* **Feature Understanding and Analysis:** Analyze summary statistics and distributions of numerical and categorical features.
* **Outlier Detection:** Detect and address outliers using statistical methods and visualization techniques.
* **Correlation Analysis:** Compute and visualize correlations between numerical features to identify relationships.
* **Trend and Pattern Analysis:** For time-related data, identify trends and seasonal patterns; analyze data grouped by categorical features.

In [ ]:
df.head()

The dataset is high-dimensional,having a total of 57 features and the target variable: "Class". Each feature describes the the frequency of words or characters in the text.
Having a dataset that is high-dimensional poses several challenges such as computational complexity, and overfitting
As a result, I will have to address this issue later in the analysis using methods such as dimensional reduction or feture selection

In [ ]:
data_type = df.dtypes
data_types_counts = data_type.value_counts()

print('Number of columns with each data type: ')
print(data_types_counts)

In [ ]:
# Summary statistics
df.describe()

In [ ]:
# Calculate skewness and kurtosis for all numerical columns
skewness = df.skew()
kurtosis = df.kurtosis()

# Create DataFrames for skewness and kurtosis
skew_df = pd.DataFrame(skewness, columns=['Skewness'])
kurtosis_df = pd.DataFrame(kurtosis, columns=['Kurtosis'])

# Concatenate skewness and kurtosis DataFrames
summary_df = pd.concat([skew_df, kurtosis_df], axis=1)

# Display the summary DataFrame
print(summary_df)

The dataset exhibits notable skewness and kurtosis in several features, indicating non-normal distributions and potential outliers. Features such as 'word_freq_3d', 'word_freq_parts', and 'char_freq_#' display particularly high positive skewness and kurtosis, suggesting heavy tails and potential outliers. Further exploration and preprocessing may be needed to address these characteristics effectively

In [ ]:
# Identify columns with missing values
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
class_counts = df['Class'].value_counts()

# Colors for the pie chart
colors = ['#66c2a5', '#fc8d62']

# Explode the second slice (1) to highlight it
explode = (0, 0.1)

# Create a pie chart
plt.figure(figsize=(5, 5))
plt.pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', startangle=140, colors=colors, explode=explode, shadow=True, wedgeprops={'linewidth': 3, 'edgecolor': 'white'})
plt.title('Distribution of Non-Fradulent and Spam emails', fontsize=16, fontweight='bold', color='navy')

# Add a legend with custom formatting
plt.legend(title='Class', loc='upper left', labels=['Non-Fraudulent (0)', 'Fraudulent (1)'], fontsize=8, bbox_to_anchor=(0.9, 0.95))

# Equal aspect ratio ensures that pie is drawn as a circle
plt.axis('equal')

# Show the plot
plt.show()

In [ ]:
# Compute the correlation matrix
correlation_matrix = df.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, fmt=".2f", linewidths=0.5)

# Add title
plt.title('Correlation Heatmap of Features')

# Show the plot
plt.show()


# **5. Data Preprocessing**

### **Key Tasks (Where Applicable)**

- **Handling Missing Values**: Identify and fill or remove missing data to ensure the dataset is complete.
- **Addressing Non-Normal Distributions**: Transform or scale features to achieve normal distribution if necessary.
- **Removing Duplicate Values**: Detect and eliminate duplicate entries to maintain data integrity.
- **Normalization**: Scale features to a standard range to ensure consistent data representation.
- **Managing Imbalanced Data**: Use techniques such as oversampling, undersampling, or synthetic data generation to balance the classes.

## 5a. Handling missing values.

In [ ]:
#### The function will return features that have missing values.(If any)

def features_with_missing_values(df):
    """
    Returns:
        > list of features that contains missing values
    
    Parameter:
        > df
    
    Returns:
        > list of features
    
    """
    return df.columns[df.isnull().any()].tolist()

missingFeatures = features_with_missing_values(df)
print(f"Features with missing values: {missingFeatures}")

The list is empty meaning, there are no missing values in the dataset

## 5b. Duplicate values.

In [ ]:
df[df.duplicated()]

In [ ]:
# Filter the DataFrame to show only duplicated rows
duplicated_rows = df[df.duplicated()]

# Count the occurrences of each class within the duplicated rows
class_distribution = duplicated_rows['Class'].value_counts()

# Display the distribution
print("Distribution of Classes within Duplicated Rows:")
print(class_distribution)

Decided to drop the duplicates as they may cause the model to overfit to the training data, as it learns patterns that are specific to the duplicates rather than general spam characteristics. This can lead to poor generalization performance on unseen data.

In [ ]:
df = df.drop_duplicates()

## 5c. Handling Imbalanced dataset.
#### Perfoming oversampling can achieve a balance between two classes

In [ ]:
# Separate the features (X) and the target variable (y)
X = df.drop('Class', axis=1)
y = df['Class']

# Instantiate SMOTE with sampling_strategy='minority' to only oversample the minority class
smote = SMOTE(sampling_strategy='minority')

# Apply SMOTE to resample the data
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
y_resampled.value_counts()

In [ ]:
#creat a new dataframe
df_clean = pd.DataFrame(X_resampled, columns=X.columns)
df_clean['Target'] = y_resampled

In [ ]:
df_clean['Target'].value_counts()

In [ ]:
class_counts = df_clean['Target'].value_counts()

# Colors for the pie chart
colors = ['#66c2a5', '#fc8d62']

# Explode the second slice (1) to highlight it
explode = (0, 0.1)

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', startangle=140, colors=colors, explode=explode, shadow=True, wedgeprops={'linewidth': 3, 'edgecolor': 'white'})
plt.title('Distribution of Classes', fontsize=16, fontweight='bold', color='navy')

# Add a legend with custom formatting
plt.legend(title='Class', loc='upper left', labels=['Not Spam (0)', 'Spam (1)'], fontsize=12, bbox_to_anchor=(0.85, 0.95))

# Equal aspect ratio ensures that pie is drawn as a circle
plt.axis('equal')

# Show the plot
plt.show()


## 5c. Handling skeweness in the data.

In [ ]:
df_clean.hist(figsize=(22,22))
plt.show()

As seen from the histograms, most of them are skewed to the left. Indicating that the distribution is highly concentrated on the lower values.

In [ ]:
# Define threshold for considering features as highly skewed
threshold = 3

# Select highly skewed features
highly_skewed_features = df_clean.columns[np.abs(df_clean.skew()) > threshold]

# Apply appropriate transformations to highly skewed features
for feature in highly_skewed_features:
    # Log transformation (adding a small constant to handle zero values)
    df_clean[feature] = np.log1p(df_clean[feature])

## 5d. Normalizing the dataset
### Normalize the datatset to equalize their influence on the model

In [ ]:
# Normalize the datatset to equalize their influence on the model

# Extract features and excluding the target var
X = df_clean.drop('Target',axis=1)

# Extract the target variable (y)
y = df_clean['Target']

scaler = StandardScaler()

# Normalize the features
X_normalized = scaler.fit_transform(X)

# Covert normalized array to df
df = pd.DataFrame(X_normalized,columns=X.columns)
df['Target'] = y

# **6. Feature Engineering**

### Key tasks:
* Create relevant features that capture key information

### Feature selection using: 
* ANOVA F-test
* Recursive Feature Elimination (RFE)
* L1-based regularization (Lasso)

In [ ]:
X = df.drop('Target',axis=1)

y= df['Target']

### 6a. Anova F-test
Higher F-test scores indicate greater importance, as these features are more likely to be informative for predicting the target variable

In [ ]:
def load_data():
    # Load your dataset
    df
    return df

def select_features_anova(X, y, k=10):
    k_best = SelectKBest(score_func=f_classif, k=k)
    k_best.fit(X, y)
    selected_indices = k_best.get_support(indices=True)
    selected_features = X.columns[selected_indices]
    plot_anova_scores(X, k_best.scores_)
    return selected_features

def plot_anova_scores(X, scores):
    plt.figure(figsize=(15, 6))
    plt.bar(range(len(scores)), scores, tick_label=X.columns)
    plt.xlabel('Features')
    plt.ylabel('ANOVA F-test score')
    plt.title('ANOVA F-test scores for feature selection')
    plt.xticks(rotation=90)
    plt.show()

def train_and_evaluate_model(X, y, selected_features):
    X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

def main():
    df = load_data()
    X = df.drop('Target', axis=1)
    y = df['Target']
    
    global selected_features_anova
    selected_features_anova = select_features_anova(X, y)
    train_and_evaluate_model(X, y, selected_features_anova)

if __name__ == "__main__":
    main()

# Accessing selected_features_anova outside the main function
print("Selected features using ANOVA F-test:", selected_features_anova)

### 6b. Recursive Feature Elimination (RFE)
RFE is an iterative feature selection method that starts with all features and recursively removes the least important features based on the coefficients of a specified machine learning model
Lower ranking values indicate higher importance, as these features were selected earlier during the RFE process

In [ ]:
def load_data():
    # Load your dataset
    df
    return df

def select_features_rfe(X, y, n_features=20):
    model = LogisticRegression()
    rfe = RFE(estimator=model, n_features_to_select=n_features)
    rfe.fit(X, y)
    selected_indices = rfe.get_support(indices=True)
    selected_features = X.columns[selected_indices]
    plot_rfe_rankings(X, rfe.ranking_)
    return selected_features

def plot_rfe_rankings(X, rankings):
    plt.figure(figsize=(15, 6))
    plt.bar(range(len(rankings)), rankings, tick_label=X.columns)
    plt.xlabel('Features')
    plt.ylabel('Ranking')
    plt.title('Feature Rankings')
    plt.xticks(rotation=90)
    plt.show()

def train_and_evaluate_model(X, y, selected_features):
    X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

def main():
    df = load_data()
    X = df.drop('Target', axis=1)
    y = df['Target']
    
    global selected_features_rfe
    selected_features_rfe = select_features_rfe(X, y)
    train_and_evaluate_model(X, y, selected_features_rfe)

if __name__ == "__main__":
    main()

# Accessing selected_features_rfe outside the main function
print("Selected features using RFE:", selected_features_rfe)

### 6c. L1-based regularization (Lasso)
The selected features are determined by their non-zero coefficients after applying L1 regularization (Lasso), prioritizing features that contribute to the model's predictive power while encouraging sparsity by shrinking some coefficients towards zero.

In [ ]:
def load_data():
    # Load your dataset
    df
    return df

def select_features_lasso(X, y, threshold='median'):
    model = LogisticRegression(penalty='l1', solver='liblinear', C=1.0)
    sfm = SelectFromModel(estimator=model, threshold=threshold)
    sfm.fit(X, y)
    selected_indices = sfm.get_support(indices=True)
    selected_features = X.columns[selected_indices]
    plot_lasso_coefficients(X, sfm, selected_indices)
    return selected_features

def plot_lasso_coefficients(X, sfm, selected_indices):
    coefficients_all = sfm.estimator_.coef_.flatten()
    coefficients_selected = coefficients_all[selected_indices]
    plt.figure(figsize=(15, 6))
    plt.plot(np.abs(coefficients_all), marker='o', linestyle='None', markersize=5, color='green', label='All Features')
    plt.plot(selected_indices, np.abs(coefficients_selected), marker='o', linestyle='None', markersize=5, color='blue', label='Selected Features')
    plt.xticks(range(len(X.columns)), X.columns, rotation=90)
    plt.xlabel('Features')
    plt.ylabel('Absolute coefficient')
    plt.title('Absolute coefficients of all features from L1 regularization (Lasso)')
    plt.legend()
    plt.grid(True)
    plt.show()

def train_and_evaluate_model(X, y, selected_features):
    X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

def main():
    df = load_data()
    X = df.drop('Target', axis=1)
    y = df['Target']
    
    global selected_features_lasso
    selected_features_lasso = select_features_lasso(X, y)
    train_and_evaluate_model(X, y, selected_features_lasso)

if __name__ == "__main__":
    main()

# Accessing selected_features_lasso outside the main function
print("Selected features using L1 regularization:", selected_features_lasso)

In [ ]:
combined_selected_features = set(selected_features_lasso) | set(selected_features_rfe) | set(selected_features_anova)

# Convert the combined selected features to a list
combined_selected_features = list(combined_selected_features)

# Display the combined selected features
print(combined_selected_features)

We combined the selected features from different feature selection techniques, including Lasso, RFE, and ANOVA, to leverage the strengths of each method and create a comprehensive feature set that captures the most relevant information for our model. By merging these sets, we aim to improve the model's predictive performance and robustness by incorporating diverse perspectives on feature importance and relevance.

## 6d. Correlation-based feature selection
It helps identify and remove highly correlated features, reducing redundancy and multicollinearity in the feature set.

In [ ]:
selected_features_with_target = combined_selected_features + ["Target"]

# Select only the combined selected features and the target variable from the DataFrame
df = df[selected_features_with_target]

corr_matrix = df.drop(columns=['Target']).corr()

# Mask for the upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Plot the heatmap
plt.figure(figsize=(22, 10))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
correlation_matrix = df.corr()

# Create a mask for highly correlated features
# The mask will be True for correlated pairs, and we'll keep the first feature
correlation_mask = (correlation_matrix.abs() > 0.5) & (correlation_matrix != 1)

# Identify columns (features) to drop
columns_to_drop = set()
for feature in correlation_mask.columns:
    correlated_features = correlation_mask.index[correlation_mask[feature]]
    if len(correlated_features) > 0:
        columns_to_drop.add(feature)

# Remove highly correlated columns from the DataFrame
df = df.drop(columns=columns_to_drop)

## Threshold Selection for Filtering Highly Correlated Features

In the analysis, I decided to set a threshold of 0.5 for filtering highly correlated features. This decision was based on several considerations:

1. **Balance between Correlation Strength and Feature Retention**: A threshold of 0.5 strikes a balance between capturing moderately strong correlations and retaining a sufficient number of features for analysis. Features with correlation coefficients above 0.5 are considered moderately correlated and may contain redundant information, which could potentially lead to multicollinearity issues in modeling.

2. **Alignment with Best Practices**: While there is no universally accepted threshold for filtering highly correlated features, a threshold of 0.5 aligns with common practices in the literature and is often used as a starting point for feature selection and dimensionality reduction tasks.

3. **Domain Knowledge**: Domain-specific knowledge plays a crucial role in determining the appropriate threshold. In this analysis, a threshold of 0.5 was deemed suitable based on an understanding of the problem domain and the relationships between the variables.

4. **Model Performance Consideration**: The selected threshold aims to strike a balance between reducing multicollinearity and preserving important information for modeling. It allows for the removal of highly correlated features that may not contribute significantly to predictive performance while retaining features with distinct information.



# **7. Model Training and Evaluation**

### Key tasks:
* Model Training
* Model Evaluation
* Reciever operating curve

In [ ]:
# Step 1: Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Target']), df['Target'], test_size=0.2, random_state=42)

# Step 2: Define models
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Step 3: Train each model
for name, model in models.items():
    model.fit(X_train, y_train)
    print(f"{name} model trained")

In [ ]:
# Step 4: Evaluate each model
metrics = {
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1 Score": []
}

best_model = None
best_accuracy = 0

for name, model in models.items():
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Store metrics
    metrics["Model"].append(name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1 Score"].append(f1)
    
    if accuracy > best_accuracy:
        best_model = model
        best_accuracy = accuracy

# Step 5: Present evaluation metrics in a table
df_metrics = pd.DataFrame(metrics)
print("\nEvaluation Metrics:")
print(df_metrics)

print(f"\nThe best model is {best_model.__class__.__name__} with test accuracy: {best_accuracy:.4f}")

In [ ]:
# Step 7: Plot ROC curve for each model
plt.figure(figsize=(12, 6))

# Random Forest
rf_fpr, rf_tpr, _ = roc_curve(y_test, models["Random Forest"].predict_proba(X_test)[:, 1])
plt.plot(rf_fpr, rf_tpr, label=f"Random Forest (AUC = {roc_auc_score(y_test, models['Random Forest'].predict_proba(X_test)[:, 1]):.2f})")

# Support Vector Machine
svm_fpr, svm_tpr, _ = roc_curve(y_test, models["Support Vector Machine"].decision_function(X_test))
plt.plot(svm_fpr, svm_tpr, label=f"Support Vector Machine (AUC = {roc_auc_score(y_test, models['Support Vector Machine'].decision_function(X_test)):.2f})")

# Logistic Regression
lr_fpr, lr_tpr, _ = roc_curve(y_test, models["Logistic Regression"].predict_proba(X_test)[:, 1])
plt.plot(lr_fpr, lr_tpr, label=f"Logistic Regression (AUC = {roc_auc_score(y_test, models['Logistic Regression'].predict_proba(X_test)[:, 1]):.2f})")

# Gradient Boosting
gb_fpr, gb_tpr, _ = roc_curve(y_test, models["Gradient Boosting"].predict_proba(X_test)[:, 1])
plt.plot(gb_fpr, gb_tpr, label=f"Gradient Boosting (AUC = {roc_auc_score(y_test, models['Gradient Boosting'].predict_proba(X_test)[:, 1]):.2f})")

# Plot ROC curve for random guessing (no skill)
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.grid(True)
plt.show()